<a href="https://colab.research.google.com/github/solanods/pandas/blob/master/ponto_segunda_quinzena.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [114]:
from google.colab import drive
drive.mount('/content/drive')

from re import search

import pandas as pd
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [115]:
df = pd.read_excel('/content/drive/MyDrive/PRMS/PRMS_01_JAN_2022.xlsm', sheet_name='RMOP')

/usr/local/lib/python3.7/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [116]:
# Exibe a planilha com todos RMOPS
df

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83,Unnamed: 84,Unnamed: 85,Unnamed: 86,Unnamed: 87,Unnamed: 88,Unnamed: 89,Unnamed: 90,Unnamed: 91,Unnamed: 92
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,ABREU,NaN,NaN,NaN,NaN,NaN,NaN,RE,957,JANEIRO,NaN,2022,Telefone:,991 015 602,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13234,B27,CovUtil12_2325,FOL,27,2022-01-27 00:00:00,FOL,FOL,_,FOL,NaN,,,FIXO,,,,NaN,NaN,NaN,NaN,NaN,_,0.0,0.0,NaN,NaN,NaN,FOL,FOL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HEFT,HE folga TARDE,hora extra vespertina em dia de folga
13235,B28,CovUtil12_2325.2,FOL,28,2022-01-28 00:00:00,FOL,FOL,_,FOL,NaN,,,FIXO,,,,NaN,NaN,NaN,NaN,NaN,_,0.0,0.0,NaN,NaN,NaN,FOL,FOL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [117]:
# Selecionar colunas de interesse
df = df.iloc[:,[3,5,6,7,8,11,16,17,18]]

In [118]:
# Renomear colunas
df = df.rename({'Unnamed: 3': 'Dia',
                'Unnamed: 5': 'Previsto',
                'Unnamed: 6': 'Escala',
                'Unnamed: 7': 'Totvs',
                'Unnamed: 8': 'Jornada',
                'Unnamed: 11': 'Entrada',
                'Unnamed: 16': 'Codigo',
                'Unnamed: 17': 'Descricao',
                'Unnamed: 18': 'Observacoes'}, axis = 1)

In [119]:
# Remover nan
df = df.fillna('-')

In [120]:
df

,Dia,Previsto,Escala,Totvs,Jornada,Entrada,Codigo,Descricao,Observacoes
0,-,-,-,-,-,-,-,-,-
1,-,-,-,-,-,-,-,-,-
2,-,-,-,-,-,-,-,-,-
3,ABREU,-,-,-,-,957,991 015 602,-,-
4,-,-,-,-,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...
13234,27,FOL,FOL,_,FOL,,-,-,-
13235,28,FOL,FOL,_,FOL,,-,-,-
13236,29,FOL,FOL,_,FOL,,-,-,-
13237,30,FOL,FOL,_,FOL,,-,-,-


In [121]:
# Mascaras da Escala
revezamento = df['Escala'].str.contains('^[A-Z]{1}\d{1}$')==True
semanal = df['Escala'].str.contains('^[A-Z]{1}\d{2}$')==True
quatro_dois_quatro = df['Escala'].str.contains('^[A-Z]{2}$')==True
aeromovel = df['Escala'].str.contains('^[A-Z]{3}$')==True
era_fg = df['Escala'].str.contains('^[A-Z]{2}\d{1}$')==True

# Mascaras da Jornada
manha = df['Jornada'].str.endswith('M')==True
tarde = df['Jornada'].str.endswith('T')==True
noite = df['Jornada'].str.endswith('N')==True
folga = df['Jornada'].str.endswith('FOL')==True

# Mascaras do Codigo
permuta = df['Codigo'].str.contains('C500')== True
troca = df['Codigo'].str.contains('C129')== True


# TROCA_PERMUTA 
df['TROCA_PERMUTA'] = ''

# Permutas
df.loc[manha & permuta, 'TROCA_PERMUTA'] = '01850'
df.loc[tarde & permuta, 'TROCA_PERMUTA'] = '01851'
df.loc[noite & permuta, 'TROCA_PERMUTA'] = '01956'
df.loc[semanal & manha & permuta, 'TROCA_PERMUTA'] = '01613'
df.loc[semanal & tarde & permuta, 'TROCA_PERMUTA'] = '01614'

# Trocas
df.loc[manha & troca, 'TROCA_PERMUTA'] = '01840'
df.loc[tarde & troca, 'TROCA_PERMUTA'] = '01841'
df.loc[noite & troca, 'TROCA_PERMUTA'] = '01845'
df.loc[folga & troca, 'TROCA_PERMUTA'] = '01768'
df.loc[semanal & manha & troca, 'TROCA_PERMUTA'] = '01610'
df.loc[semanal & tarde & troca, 'TROCA_PERMUTA'] = '01611'
df.loc[semanal & folga & troca, 'TROCA_PERMUTA'] = '01612'

# ESCALA_TOTVS
df['ESCALA_TOTVS'] = ''

# Mascaras da Escala
revezamento_manha = df['Escala'].str.contains('^[A-I]{1}\d{1}$')==True
revezamento_tarde = df['Escala'].str.contains('^[J-R]{1}\d{1}$')==True
semanal_manha = df['Escala'].str.startswith('X')==True
semanal_tarde = df['Escala'].str.startswith('Y')==True
quatro_dois_quatro_manha = df['Escala'].str.contains('^[A-I][A-B]$')==True
quatro_dois_quatro_tarde = df['Escala'].str.contains('^[J-R][A-B]$')==True
aeromovel_manha = df['Escala'].str.contains('^[A-E][A-Z]{2}$')==True
aeromovel_tarde = df['Escala'].str.contains('^[J-N][A-Z]{2}$')==True


df.loc[revezamento_manha, 'ESCALA_TOTVS'] = '01116'
df.loc[revezamento_tarde, 'ESCALA_TOTVS'] = '01729'
df.loc[semanal_manha, 'ESCALA_TOTVS'] = '01600'
df.loc[semanal_tarde, 'ESCALA_TOTVS'] = '01601'
df.loc[quatro_dois_quatro_manha, 'ESCALA_TOTVS'] = '01622'
df.loc[quatro_dois_quatro_tarde, 'ESCALA_TOTVS'] = '01630'
df.loc[aeromovel_manha, 'ESCALA_TOTVS'] = '01799'
df.loc[aeromovel_tarde, 'ESCALA_TOTVS'] = '01811'

In [122]:
  # Obter lista com nome de todos operadores
  operadores = []
  for linha in df.index:
    if search ('[A-Z]+', str(df['Dia'][linha])):
      operadores.append(df['Dia'][linha])

  operadores = [operador for operador in operadores if operador.isupper()]

In [123]:
# Extrair rmop
i = 3
j =38
RMOP = []
for operador in operadores:
  rmop_operador = df.loc[i:j]
  RMOP.append(rmop_operador)
  i+=66
  j+=66

In [124]:
# Exibir índices dos RMOPS dos operadores
for operador in operadores:
  print(operador, 'RMOP[', operadores.index(operador), ']')

ABREU RMOP[ 0 ]
ADAO RMOP[ 1 ]
AG-HUR RMOP[ 2 ]
AGNALDO RMOP[ 3 ]
ALBARELLO RMOP[ 4 ]
ALBERTO RMOP[ 5 ]
ALEXSANDRO RMOP[ 6 ]
ALMEIDA RMOP[ 7 ]
ALVES RMOP[ 8 ]
AMON RMOP[ 9 ]
ANA CAROLINA RMOP[ 10 ]
ANATOLI RMOP[ 11 ]
ANDRADE RMOP[ 12 ]
ANDRE RMOP[ 13 ]
ANDREI RMOP[ 14 ]
ANTONIO RMOP[ 15 ]
ARAUJO RMOP[ 16 ]
ARRUE RMOP[ 17 ]
ARTUR RMOP[ 18 ]
AZAMBUJA RMOP[ 19 ]
BACK RMOP[ 20 ]
BARCELLOS RMOP[ 21 ]
BEATRIZ RMOP[ 22 ]
BEN-HUR RMOP[ 23 ]
BENHUR DA SILVA RMOP[ 24 ]
BOLZAN RMOP[ 25 ]
CARBONERA RMOP[ 26 ]
CAREN BRAGA RMOP[ 27 ]
CARLOS RMOP[ 28 ]
CARMEN RMOP[ 29 ]
CESAR RMOP[ 30 ]
CHAGAS RMOP[ 31 ]
CHAVES RMOP[ 32 ]
CIBELE RMOP[ 33 ]
CLAITON RMOP[ 34 ]
CLAUDETE RMOP[ 35 ]
CLAUDETE MOURA RMOP[ 36 ]
CLAUDIA RMOP[ 37 ]
CLOVIS RMOP[ 38 ]
COITINHO RMOP[ 39 ]
CORREA RMOP[ 40 ]
COSTA RMOP[ 41 ]
CRISTIANO RMOP[ 42 ]
DA COSTA RMOP[ 43 ]
DADDA RMOP[ 44 ]
DAL'AGO RMOP[ 45 ]
DIAS RMOP[ 46 ]
DIEGO RMOP[ 47 ]
DIEGO SILVA RMOP[ 48 ]
DIMAS RMOP[ 49 ]
DORNELLES RMOP[ 50 ]
DUARTE RMOP[ 51 ]
EDNA RMOP[ 52 ]
ELIES

In [125]:
# Exibir RMOP do operador
RMOP[112]

,Dia,Previsto,Escala,Totvs,Jornada,Entrada,Codigo,Descricao,Observacoes,TROCA_PERMUTA,ESCALA_TOTVS
7395,MADALENA,-,-,-,-,972,997 519 683,-,-,,
7396,-,-,-,-,-,-,-,-,-,,
7397,Folgas:,Prev,-,4,Real,-,-,-,-,,
7398,Dia,Escala,-,-,-,-,Código,-,Observações,,
7399,-,PREV,-,TOTVS,Jor,Entrada,Nº,Descrição,-,,
7400,1,FER,FER,_,FER,,-,-,-,,
7401,2,FER,FER,_,FER,,-,-,-,,
7402,3,FER,FER,_,FER,,-,-,-,,
7403,4,FER,FER,_,FER,,-,-,-,,
7404,5,FER,FER,_,FER,,-,-,-,,


In [126]:
# Primeira quinzena
RMOP[112].iloc[:20,:]

,Dia,Previsto,Escala,Totvs,Jornada,Entrada,Codigo,Descricao,Observacoes,TROCA_PERMUTA,ESCALA_TOTVS
7395,MADALENA,-,-,-,-,972,997 519 683,-,-,,
7396,-,-,-,-,-,-,-,-,-,,
7397,Folgas:,Prev,-,4,Real,-,-,-,-,,
7398,Dia,Escala,-,-,-,-,Código,-,Observações,,
7399,-,PREV,-,TOTVS,Jor,Entrada,Nº,Descrição,-,,
7400,1,FER,FER,_,FER,,-,-,-,,
7401,2,FER,FER,_,FER,,-,-,-,,
7402,3,FER,FER,_,FER,,-,-,-,,
7403,4,FER,FER,_,FER,,-,-,-,,
7404,5,FER,FER,_,FER,,-,-,-,,


In [127]:
# Segunda quinzena
RMOP[112].iloc[20:,:]

,Dia,Previsto,Escala,Totvs,Jornada,Entrada,Codigo,Descricao,Observacoes,TROCA_PERMUTA,ESCALA_TOTVS
7415,16,FER,FER,_,FER,,-,-,-,,
7416,17,FER,FER,_,FER,,-,-,-,,
7417,18,TRAB,ASS,_,ASS,,-,-,-,,01799
7418,19,TRAB,ASS,_,ASS,,-,-,-,,01799
7419,20,TRAB,ASS,_,ASS,,-,-,-,,01799
7420,21,TRAB,Y10,_,07T,13:15:00,C083,Troca Missão,Ajuste de efetivo,,01601
7421,22,FOL,Y10,_,07T,12:40:00,HEFT,HE folga TARDE,Necessidade Operacional,,01601
7422,23,FOL,Y10,_,FOL,,-,-,-,,01601
7423,24,TRAB,Y10,_,26T,12:10:00,-,-,-,,01601
7424,25,TRAB,Y10,_,26T,12:10:00,C083,Troca Missão,c/Freitas,,01601


In [128]:
RMOP[86]

,Dia,Previsto,Escala,Totvs,Jornada,Entrada,Codigo,Descricao,Observacoes,TROCA_PERMUTA,ESCALA_TOTVS
5679,JONAS,-,-,-,-,2882,991 999 452,-,-,,
5680,-,-,-,-,-,-,-,-,-,,
5681,Folgas:,Prev,-,7,Real,-,-,-,-,,
5682,Dia,Escala,-,-,-,-,Código,-,Observações,,
5683,-,PREV,-,TOTVS,Jor,Entrada,Nº,Descrição,-,,
5684,1,FER,FER,_,FER,,-,-,-,,
5685,2,FER,FER,_,FER,,-,-,-,,
5686,3,FER,FER,_,FER,,-,-,-,,
5687,4,FER,FER,_,FER,,-,-,-,,
5688,5,FER,FER,_,FER,,-,-,-,,


In [130]:
df['ESCALA_TOTVS'].value_counts()

         9165
01729    1360
01116    1285
01799     302
01630     274
01811     261
01622     248
01601     204
01600     140
Name: ESCALA_TOTVS, dtype: int64

In [131]:
df['TROCA_PERMUTA'].value_counts()

         12589
01768      263
01841       76
01840       76
01851       63
01956       62
01850       53
01845       44
01612        6
01611        3
01613        3
01610        1
Name: TROCA_PERMUTA, dtype: int64